In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from tqdm import tqdm
import pickle
tqdm.pandas()
import warnings
import catboost as cb
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
from submission_xgboost.utils import *
from sklearn.metrics import r2_score

In [2]:
from submission_xgboost.utils import  propagate_orbit

### Load Initial position and dict of Omni

In [3]:
initial_states = pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/00000_to_02284-initial_states.csv')
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/02285_to_02357-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/02358_to_04264-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/04265_to_05570-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/05571_to_05614-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/05615_to_06671-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/06672_to_08118-initial_states.csv')])
initial_states['Timestamp'] = pd.to_datetime(initial_states['Timestamp'])
initial_states = initial_states.set_index('File ID')
print(initial_states.shape)
initial_states =  initial_states.mask((initial_states==9.99e+29)|(initial_states==9.99e+32))
initial_states = initial_states.loc[initial_states.count(1)==10]
print(initial_states.shape)

(8119, 10)
(8095, 10)


In [4]:
initial_states['DOY'] = initial_states['Timestamp'].dt.day_of_year
initial_states['HOUR'] = initial_states['Timestamp'].dt.hour
initial_states['Year'] = initial_states['Timestamp'].dt.year
initial_states['bartels_phase'] = (initial_states['DOY'] % 27)/27 * 360

### Estimates lat, long, alt

In [ ]:
model_alt = cb.CatBoostRegressor(loss_function='RMSE',iterations=100,subsample=0.7)
model_alt.fit(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[:4000],
              y= initial_states.dropna()['Altitude (km)'].iloc[:4000])

initial_states['Altitude (km) estimated'] = model_alt.predict(initial_states[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']])

In [ ]:
initial_states['Altitude (km) estimated2'] = (initial_states['Semi-major Axis (km)'] * (1-initial_states['Eccentricity']**2)/(1+initial_states['Eccentricity']*initial_states['True Anomaly (deg)'].apply(np.cos))) -  6378.137      

In [ ]:
a * (1 - e**2) / (1 + e * np.cos(nu))

In [ ]:
model_alt.score(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[4000:],
              y= initial_states.dropna()['Altitude (km)'].iloc[4000:])

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,3))
initial_states.iloc[4000:].plot.scatter('Altitude (km) estimated2','Altitude (km)',ax=ax[0])
ax[0].set_title('R2 on test: 0.52')
initial_states.iloc[4000:].plot.scatter('Latitude (deg) estimated','Latitude (deg)',ax=ax[1])
ax[1].set_title('R2 on test: 0.25')
initial_states.iloc[4000:].plot.scatter('Longitude (deg) estimated','Longitude (deg)',ax=ax[2])
ax[2].set_title('R2 on test: -0.04')

In [ ]:
omni[0
]

In [ ]:
model_alt.save_model('data/trained_model/Altitude_Model.cbm')

In [ ]:
model_lat = cb.CatBoostRegressor(loss_function='RMSE',iterations=1000,subsample=0.8,depth=2)
model_lat.fit(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[:4000],
              y= initial_states.dropna()['Latitude (deg)'].iloc[:4000])
initial_states['Latitude (deg) estimated'] = model_lat.predict(initial_states[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']])

In [ ]:
model_lat.score(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[4000:],
              y= initial_states.dropna()['Latitude (deg)'].iloc[4000:])

In [ ]:
model_lat.save_model('data/trained_model/Latitude_Model.cbm')

In [ ]:
model_long = cb.CatBoostRegressor(loss_function='RMSE',iterations=100,depth=2)
model_long.fit(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[:4000],
              y= initial_states.dropna()['Longitude (deg)'].iloc[:4000])
initial_states['Longitude (deg) estimated'] = model_long.predict(initial_states[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']])

In [ ]:
model_long.score(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[4000:],
              y= initial_states.dropna()['Longitude (deg)'].iloc[4000:])

In [ ]:
model_long.save_model('data/trained_model/Longitude_Model.cbm')

### Read data

In [ ]:
# all_files_omni = os.listdir("phase_1/omni2/")
# all_files_dens = os.listdir("phase_1/sat_density/")
# omni = {}
# for i in tqdm(all_files_omni):
#     omni[int(i.split('-')[1])] = pd.read_csv('phase_1/omni2/'+i)
# sat = {}
# for i in tqdm(all_files_dens):
#     sat[int(i.replace('gr-of','grof').split('-')[1])] = pd.read_csv('phase_1/sat_density/'+i)
# with open('omni.pickle', 'wb') as handle:
#     pickle.dump(omni, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('sat.pickle', 'wb') as handle:
#     pickle.dump(sat, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
with open('data/data_isaac/omni.pickle', 'rb') as f:
    omni = pickle.load(f)
with open('data/data_isaac/sat.pickle', 'rb') as f:
    sat = pickle.load(f)

for i in sat.keys():
    sat[i]['Orbit Mean Density (kg/m^3)'] = sat[i]['Orbit Mean Density (kg/m^3)'].mask(sat[i]['Orbit Mean Density (kg/m^3)']>1_000_000)
    sat[i]['Orbit Mean Density (kg/m^3)'] = sat[i]['Orbit Mean Density (kg/m^3)'].interpolate()
    #sat[i]['Orbit Mean Density (kg/m^3)'] = sat[i]['Orbit Mean Density (kg/m^3)'].ffill().bfill() # add forward and backward fill

In [6]:
len_sat = {i:len(sat[i]['Orbit Mean Density (kg/m^3)'].dropna()) for i in sat.keys()}
initial_states['len'] = pd.Series(len_sat)
initial_states = initial_states.loc[initial_states['len'].isin([432,433])]
initial_states = initial_states.drop('len',axis=1)
initial_states.shape

(7870, 14)

In [7]:
del omni, sat

### Compute R2 with mean and median

In [ ]:
all_sat = pd.concat(sat,keys=sat.keys())
all_sat = all_sat.reset_index().drop(['level_1'],axis=1).rename({'level_0':'File Id'},axis=1)
df_mean = all_sat.groupby('File Id')['Orbit Mean Density (kg/m^3)'].mean()
df_median = all_sat.groupby('File Id')['Orbit Mean Density (kg/m^3)'].median()
all_sat = pd.merge_ordered(all_sat,df_mean,on='File Id',suffixes=['','_mean'])
all_sat = pd.merge_ordered(all_sat,df_median,on='File Id',suffixes=['','_median'])

In [ ]:
a = all_sat.groupby('File Id').apply(lambda win: r2_score(win.dropna()['Orbit Mean Density (kg/m^3)'], win.dropna()['Orbit Mean Density (kg/m^3)_mean']) if len(win.dropna())>0 else np.nan)

In [ ]:
b = all_sat.loc[all_sat['File Id']==101].dropna()

In [ ]:
r2_score(b['Orbit Mean Density (kg/m^3)']*10**12,b['Orbit Mean Density (kg/m^3)_mean']*10**12)

In [ ]:
b['Orbit Mean Density (kg/m^3)'].plot()
b['Orbit Mean Density (kg/m^3)_mean'].plot()

In [ ]:
b

In [ ]:
a.head(50)

In [ ]:
r2_score(all_sat.dropna()['Orbit Mean Density (kg/m^3)'], all_sat.dropna()['Orbit Mean Density (kg/m^3)_mean'])

In [ ]:
r2_score(all_sat.dropna()['Orbit Mean Density (kg/m^3)'], all_sat.dropna()['Orbit Mean Density (kg/m^3)_median'])

In [ ]:
id = 100
all_sat['Orbit Mean Density (kg/m^3)'].loc[all_sat['File Id']==id].plot()
all_sat['Orbit Mean Density (kg/m^3)_mean'].loc[all_sat['File Id']==id].plot()
all_sat['Orbit Mean Density (kg/m^3)_median'].loc[all_sat['File Id']==id].plot()

### See features

In [ ]:
plt.scatter(df_data['Target'],df_data['Altitude (km)'])

In [ ]:
df_data['Inclination (deg)'].hist(bins=500)

In [ ]:
plt.scatter(df_data['Target'],df_data['mean_f10.7_index'])

In [ ]:
plt.scatter(df_data['Target'],np.exp(df_data['min_f10.7_index']))

In [ ]:
df_data.corr()['Target'].abs().sort_values(ascending=False).head(50)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,2))
a.set_index('Timestamp')['Alfen_mach_number'].interpolate().plot(color='Red',label='Interpolated data',ax=ax[0])
a.set_index('Timestamp')['Alfen_mach_number'].plot(color='green',label='Alfen_mach_number',ax = ax[0])
ax[0].legend()

a.set_index('Timestamp')['SW_Plasma_Temperature_K'].interpolate().plot(color='Red',label='Interpolated data',ax=ax[1])
a.set_index('Timestamp')['SW_Plasma_Temperature_K'].plot(color='green',label='SW_Plasma_Temperature_K',ax = ax[1])
ax[1].legend()



a.set_index('Timestamp')['Quasy_Invariant'].interpolate().plot(color='Red',label='Interpolated data',ax=ax[2])
a.set_index('Timestamp')['Quasy_Invariant'].plot(color='green',label='Quasy_Invariant',ax = ax[2])
ax[2].legend()

                

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(6,3))
pd.concat([omni[i]['SW_Proton_Density_N_cm3'] for i in omni.keys()]).hist(bins=100,ax=ax[0])
ax[0].set_title('Raw data')
pd.concat([omni[i]['SW_Proton_Density_N_cm3'].replace(999.9,np.nan) for i in omni.keys()]).hist(bins=100,ax=ax[1])
ax[1].set_title('After correction')
ax[0].set_ylabel('SW_Proton_Density_N_cm3')

In [ ]:
name = 'Alpha_Prot_ratio'
fig,ax = plt.subplots(1,4,figsize=(16,2))
pd.concat([omni[i][name] for i in omni.keys()]).hist(bins=100,ax=ax[0])
ax[0].set_title(name + '\nRaw data')
pd.concat([omni[i][name].replace(incorrect_values[name],np.nan) for i in omni.keys()]).hist(bins=100,ax=ax[1])
ax[1].set_title(name+'\nAfter correction (NaN values: '+ str(incorrect_values[name]) + ')')

name_2 = 'SW_Proton_Density_N_cm3'
pd.concat([omni[i][name_2] for i in omni.keys()]).hist(bins=100,ax=ax[2],color='red')
ax[2].set_title(name_2 + '\nRaw data')
pd.concat([omni[i][name_2].replace(incorrect_values[name_2],np.nan) for i in omni.keys()]).hist(bins=100,ax=ax[3],color='red')
ax[3].set_title(name_2+'\nAfter correction (NaN values: '+ str(incorrect_values[name_2]) + ')')


### Prepare data

In [ ]:
# res = {}
# for i in tqdm(initial_states.index):
#     res[i] = create_features_and_target(i,initial_states,omni = omni[i],sat = sat[i],predict_mean=True,training_mode=True)
# with open('data/data_isaac/res_extented_8000.pickle', 'wb') as handle:
#      pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
with open('data/data_isaac/res_extented_8000_with_proj_all.pickle', 'rb') as f:
    res = pickle.load(f)

## XGBOOST

### Prediction by mean

In [ ]:
df_data = pd.concat(res)
#df_data = df_data.reset_index().rename({'level_0':'File Id'},axis=1)

In [ ]:
index = list(res.keys())
np.random.seed(24)
np.random.shuffle(index)

In [ ]:
4600/8000

In [ ]:
train_index,valid_index,test_index = index[:4600],index[4600:5900],index[5900:]
col_to_remove = ['Timestamp_initial','File Id']
print(len(col_to_remove))

In [ ]:
# target_columns = ['Target_0','Target_1','Target_2','Target_3','Target_4','Target_5','Target_mean']
# target_columns_1 = ['Target_mean']
# target_columns_2 = ['Target_0','Target_1','Target_2','Target_3','Target_4','Target_5']
target_columns = ['Target_mean']
target_columns_1 = ['Target_mean']

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('C:/Users/isaac/Downloads/model_by_mean_expanded_xgboost_with/model_by_mean_expanded_xgboost.cbm')

#### All train

In [ ]:
df_data = df_data.set_index('File Id').loc[list( set(initial_states.index))].reset_index()
df_data = df_data.drop(col_to_remove,axis=1)
df_data = df_data.dropna(subset=target_columns)

train_dataset_all = cb.Pool(df_data.drop(target_columns,axis=1), df_data[target_columns]* 10**12) 
print(df_training_set_all.shape)

In [ ]:
model_mean = cb.CatBoostRegressor(loss_function='RMSE',iterations=1000)
model_mean.fit(train_dataset_all)

In [ ]:
df_data.columns[:50]

In [ ]:
get_model_feat_import(model_mean,30)

In [ ]:
model_mean.score(train_dataset_all)

In [ ]:
model_mean.save_model('data/trained_model/model_by_mean_new_test.cbm')

#### Train/Valid/Test

In [ ]:
df_data['Target_mean']

In [ ]:
df_training_set = df_data.set_index('File Id').loc[list(set(train_index) & set(initial_states.index))].reset_index()
df_training_set = df_training_set.drop(col_to_remove,axis=1)
df_training_set = df_training_set.dropna(subset=target_columns)

df_valid_set = df_data.set_index('File Id').loc[list(set(valid_index) & set(initial_states.index))].reset_index()
df_valid_set = df_valid_set.drop(col_to_remove,axis=1)
df_valid_set = df_valid_set.dropna(subset=target_columns)

df_test_set = df_data.set_index('File Id').loc[list(set(test_index) & set(initial_states.index))].reset_index()
df_test_set = df_test_set.drop(col_to_remove,axis=1)
df_test_set = df_test_set.dropna(subset=target_columns)

print(df_training_set.shape)
print(df_valid_set.shape)
print(df_test_set.shape)

#### Task 1

In [ ]:
train_dataset_task_1 = cb.Pool(df_training_set.drop(target_columns,axis=1), df_training_set[target_columns_1]* 10**13) 
valid_dataset_task_1 = cb.Pool(df_valid_set.drop(target_columns,axis=1), df_valid_set[target_columns_1]* 10**13) 
test_dataset_task_1 = cb.Pool(df_test_set.drop(target_columns,axis=1), df_test_set[target_columns_1]* 10**13) 

In [ ]:
model_task_1 = cb.CatBoostRegressor(loss_function='RMSE',iterations=10000)
model_task_1.fit(train_dataset_task_1,eval_set=test_dataset_task_1)

In [ ]:
print(model_task_1.score(train_dataset_task_1))
print(model_task_1.score(valid_dataset_task_1))
print(model_task_1.score(test_dataset_task_1))

In [ ]:
df_data['Avg_Prediction'] = model_task_1.predict(df_data)/10**13

In [ ]:
all_sat = pd.merge(all_sat,df_data[['Avg_Prediction']].reset_index(),left_on='File Id',right_on='index')

In [ ]:
a = all_sat.groupby('File Id').apply(lambda win: r2_score(win.dropna()['Orbit Mean Density (kg/m^3)'], win.dropna()['Avg_Prediction']) if len(win.dropna())>0 else np.nan)

In [ ]:
all_sat.loc[all_sat['File Id']==115][['Orbit Mean Density (kg/m^3)','Orbit Mean Density (kg/m^3)_mean','Avg_Prediction'	]].plot()

In [ ]:
a.loc[a>-30].hist(bins=100,figsize=(10))


In [ ]:
a

In [ ]:
a.min()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(19,2))
df_data.set_index('File Id').loc[list(set(train_index) & set(initial_states.index))].reset_index().plot.scatter('Target_mean','Avg_Prediction',ax=ax[0],label='Train')
df_data.set_index('File Id').loc[list(set(valid_index) & set(initial_states.index))].reset_index().plot.scatter('Target_mean','Avg_Prediction',ax=ax[0],color='red',label='Valid')
df_data.set_index('File Id').loc[list(set(test_index) & set(initial_states.index))].reset_index().plot.scatter('Target_mean','Avg_Prediction',ax=ax[0],color='green',label='Test')
plt.legend()

a.loc[a>-30].hist(bins=100,ax=ax[1])
ax[1].set_title('Hisogram of R2')

all_sat.loc[all_sat['File Id']==115][['Orbit Mean Density (kg/m^3)','Orbit Mean Density (kg/m^3)_mean','Avg_Prediction'	]].plot(ax=ax[2])
ax[2].set_title('Example of prediction on initial state: 115')

In [ ]:
model_task_1.save_model('data/trained_model/model_with_alt_proj.cbm')

#### Task 2

In [ ]:
df_training_set['estimated'] = model_task_1.predict(df_training_set)
#df_valid_set['estimated'] = model_task_1.predict(df_valid_set)
df_test_set['estimated'] = model_task_1.predict(df_test_set)

In [ ]:
df_data['estimated'] = model_task_1.predict(df_data)

In [ ]:
train_dataset_task_2 = cb.Pool(df_training_set.drop(target_columns,axis=1), df_training_set[target_columns_2]* 10**13) 
#valid_dataset_task_2 = cb.Pool(df_valid_set.drop(target_columns,axis=1), df_valid_set[target_columns_2]* 10**13) 
test_dataset_task_2 = cb.Pool(df_test_set.drop(target_columns,axis=1), df_test_set[target_columns_2]* 10**13) 

In [ ]:
model_task_2 = cb.CatBoostRegressor(loss_function='MultiRMSE',iterations=1000)
model_task_2.fit(train_dataset_task_2,eval_set=test_dataset_task_2)

In [ ]:
print(model_task_2.score(train_dataset_task_2))
#print(model_task_2.score(valid_dataset_task_2))
print(model_task_2.score(test_dataset_task_2))

In [ ]:
get_model_feat_import(model_task_2)

In [ ]:
# pred = []
# for tar in target_columns_2:
#     train_dataset_task_2 = cb.Pool(df_training_set.drop(target_columns,axis=1), df_training_set[tar]* 10**12) 
#     valid_dataset_task_2 = cb.Pool(df_valid_set.drop(target_columns,axis=1), df_valid_set[tar]* 10**12) 
#     test_dataset_task_2 = cb.Pool(df_test_set.drop(target_columns,axis=1), df_test_set[tar]* 10**12) 
    
#     model_task_2 = cb.CatBoostRegressor(loss_function='RMSE',iterations=1000)
#     model_task_2.fit(train_dataset_task_2,eval_set=valid_dataset_task_2)

#     pred.append(pd.DataFrame(model_task_2.predict(df_data),columns = [tar])/10**12)
#     print(tar)
#     print(model_task_2.score(train_dataset_task_2))
#     print(model_task_2.score(valid_dataset_task_2))
#     print(model_task_2.score(test_dataset_task_2))

In [ ]:
def do_interpolation(result):
    df = pd.Series(result,index=range(36,432,72)).to_frame()
    return df.reindex(range(0,432)).interpolate()

In [ ]:
df_prediction_task_1 = pd.Series(model_task_1.predict(df_data), index = df_data['File Id'])/10**13
df_prediction_task_2 = pd.DataFrame(model_task_2.predict(df_data), index = df_data['File Id'])/10**13
df_prediction = df_prediction_task_2.add(df_prediction_task_1,axis=0)

df_prediction_interpolated = pd.concat(df_prediction.apply(lambda win:do_interpolation(win.values),axis=1).values,axis=1).T
df_prediction_interpolated.index = df_data['File Id']
df_prediction_interpolated = df_prediction_interpolated.T.bfill().T

df_prediction_mean = pd.concat([df_prediction_task_1] * 432, axis=1)
df_prediction_mean.index = df_data['File Id']

df_true = pd.concat({i:sat[i]['Orbit Mean Density (kg/m^3)'] for i in df_data['File Id'].values},axis=1).T

In [ ]:
df_true = pd.concat({i:sat[i]['Orbit Mean Density (kg/m^3)'] for i in df_data['File Id'].values},axis=1).T

In [ ]:
df_true.max().quantile(0.95)

In [ ]:
all_r2_mean = {}
all_r2_interpolated = {}
for i in df_prediction_interpolated.index:
    all_r2_mean[i] = r2_score(df_true.loc[i].iloc[:432].bfill(),df_prediction_mean.loc[i].iloc[:432])
    all_r2_interpolated[i] = r2_score(df_true.loc[i].iloc[:432].bfill(),df_prediction_interpolated.loc[i].iloc[:432])

r2_all = pd.concat([pd.Series(all_r2_mean),pd.Series(all_r2_interpolated),initial_states],axis=1).sort_values(1)
r2_all.columns = ['R2_Mean','R2_Interpolated'] + list(initial_states.columns)
r2_all.loc[train_index,'Cat'] = 'Train'
r2_all.loc[valid_index,'Cat'] = 'Valid'
r2_all.loc[test_index,'Cat'] = 'Test'

In [ ]:
# print(pd.Series(all_r2_mean).loc[train_index].mean())
# print(pd.Series(all_r2_interpolated).loc[train_index].mean())

# print(pd.Series(all_r2_mean).loc[valid_index].mean())
# print(pd.Series(all_r2_interpolated).loc[valid_index].mean())

# print(pd.Series(all_r2_mean).loc[test_index].mean())
# print(pd.Series(all_r2_interpolated).loc[test_index].mean())

In [ ]:
index = 5190                                 
result = model_task_1.predict(df_data.loc[df_data['File Id']==index])/10**13

# Computed
prediction = df_prediction_interpolated.loc[[index]].T
prediction.columns = ["Computed"]
prediction['Mean'] = df_prediction_task_1.loc[index]
prediction.plot(label='Computed')

#True
plt.scatter(range(36,432,72),res[index][target_columns_1].values+res[index][target_columns_2].values,color='red')
sat[index]['Orbit Mean Density (kg/m^3)'].plot(label='True')
plt.legend()
print('R2 with mean: ',r2_score(sat[index]['Orbit Mean Density (kg/m^3)'].iloc[:432],prediction['Mean'].bfill()))
print('R2 with interpolated: ',r2_score(sat[index]['Orbit Mean Density (kg/m^3)'].iloc[:432],prediction['Computed'].bfill()))

In [ ]:
model_task_1.save_model('data/trained_model/model_by_mean_task_1.cbm')
model_task_2.save_model('data/trained_model/model_by_mean_task_2.cbm')

### Prediction by timestamp to predict

#### Test train validate

In [ ]:
index = list(res.keys())
np.random.seed(24)
np.random.shuffle(index)

In [ ]:
#train_index,valid_index,test_index = index[:5600],index[5600:6900],index[6900:]

In [ ]:
train_index,valid_index,test_index = index[4000:4600],index[4600:5900],index[5900:]
col_to_remove = ['FileId','Timestamp','Timestamp_initial'] +\
                [i for i in res[0].columns if i.find('YEAR')!=-1]+\
                [i for i in res[0].columns if i.find('DOY')!=-1]+\
                [i for i in res[0].columns if i.find('Hour')!=-1]+\
                [i for i in res[0].columns if i.find('std_')!=-1]+\
                [i for i in res[0].columns if i.find('median_')!=-1]
                #[i for i in res[0].columns if i.find('mean_')!=-1]+\
                #[i for i in res[0].columns if i.find('max_')!=-1]+\
                #[i for i in res[0].columns if i.find('min_')!=-1]+\
print(len(col_to_remove))

In [ ]:
df_training_set = pd.concat( dict((k,res[k]) for k in train_index if k in initial_states.index)).reset_index().rename({'level_0':'FileId'},axis=1)
df_training_set = df_training_set.drop(col_to_remove,axis=1)
df_training_set = df_training_set.dropna(subset=['Orbit Mean Density (kg/m^3)'],axis=0)

df_valid_set =  pd.concat( dict((k,res[k]) for k in valid_index if k in initial_states.index)).reset_index().rename({'level_0':'FileId'},axis=1)
df_valid_set = df_valid_set.drop(col_to_remove,axis=1)
df_valid_set = df_valid_set.dropna(subset=['Orbit Mean Density (kg/m^3)'],axis=0)

df_test_set =  pd.concat( dict((k,res[k]) for k in test_index if k in initial_states.index)).reset_index().rename({'level_0':'FileId'},axis=1)
df_test_set = df_test_set.drop(col_to_remove,axis=1)
df_test_set = df_test_set.dropna(subset=['Orbit Mean Density (kg/m^3)'],axis=0)

train_dataset = cb.Pool(df_training_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_training_set['Orbit Mean Density (kg/m^3)']* 10**12) 
valid_dataset = cb.Pool(df_valid_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_valid_set['Orbit Mean Density (kg/m^3)']* 10**12) 
test_dataset = cb.Pool(df_test_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_test_set['Orbit Mean Density (kg/m^3)']* 10**12) 

In [ ]:
print(df_training_set.shape)
print(df_valid_set.shape)
print(df_test_set.shape)

#### All 

In [ ]:
df_training_set = pd.concat(res.values).reset_index()

df_training_set = df_training_set.reset_index().rename({'index':'FileId'},axis=1)
df_training_set = df_training_set.drop(col_to_remove,axis=1)
df_training_set = df_training_set.dropna(subset=['Orbit Mean Density (kg/m^3)'],axis=0)

train_dataset = cb.Pool(df_training_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_training_set['Orbit Mean Density (kg/m^3)']* 10**12) 

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE',iterations=360,depth=6,eta=0.2,subsample=0.9,l2_leaf_reg=1)
model.fit(train_dataset)

In [ ]:
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
data.head(15)

In [ ]:
print(model.score(train_dataset))

In [ ]:
model.save_model('data/trained_model/model_by_timestamp_xgboost_optimized_parameters_2.cbm')

#### Train model

In [ ]:
model = cb.CatBoostRegressor(iterations=100)
grid = {'subsample': [ 0.8, 0.9],
        'depth': [2,4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
       }
grid_search_result  = model.grid_search(grid, X = train_dataset)

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE',iterations=260,depth=2,eta=0.2,subsample=0.9,l2_leaf_reg=1)
model.fit(train_dataset,eval_set=valid_dataset)

In [ ]:
print(model.score(train_dataset))
print(model.score(valid_dataset))
print(model.score(test_dataset))

In [ ]:
plt.hist(model.predict(train_dataset)/10**12,bins=200)
df_training_set['Orbit Mean Density (kg/m^3)'].hist(bins=200,alpha=0.5)

In [ ]:
pd.DataFrame(train_dataset.get_label()).iloc[:10000:,0].plot(label='Train label')
pd.DataFrame(model.predict(train_dataset)).iloc[:10000,0].plot(label='Model prediction')
plt.legend()

In [ ]:
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
                     'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False) 
data.head(20)

In [ ]:
model.save_model('data/trained_model/model_by_timestamp_xgboost_second.cbm')

In [ ]:
id = 101
res[id].reset_index()['Orbit Mean Density (kg/m^3)'].plot()
plt.plot(model.predict(res[id])/10**12,color='red')





###  Compare pred

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('data/trained_model/model_by_timestamp_xgboost_test.cbm')
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
data.head(15)

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('data/trained_model/model_by_timestamp_xgboost_26_02_2025.cbm')
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
data.head(15)

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('data/trained_model/model_by_timestamp.cbm')
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
data.head(15)

In [ ]:
# df_pred_by_mean = pd.DataFrame(create_pred_timestamps(df_train['Timestamp'].loc[id]),columns=['Timestamp'])
# df_pred_by_mean['Orbit Mean Density (kg/m^3)'] = model_mean.predict(create_features_and_target_by_dict(id,df_train,omni,sat).drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp','Target']))

In [ ]:
id = 10


In [ ]:
pred_feature = create_features_and_target(id,initial_states,omni,sat,predict_mean=False)
df_pred = pd.DataFrame(model.predict(pred_feature.reset_index().drop(['Timestamp'],axis=1)),
             index=pred_feature.index,columns=['Orbit Mean Density (kg/m^3)']).reset_index()

In [ ]:
df_pred

In [ ]:
df_pred_msis = pd.read_csv('data/data_isaac/sat_density_pred/density_pred'+str(id)+ '.csv',index_col=0)
df_pred_msis['Timestamp'] = pd.to_datetime(df_pred_msis['Timestamp'])

In [ ]:
df_pred_msis.shape

In [ ]:
df_true = sat[id]

In [ ]:
plt.figure()
df_true.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
df_pred.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
#df_pred_by_mean.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
df_pred_msis.set_index('Timestamp')['Density (kg/m3)'].plot()

In [ ]:
df_true

In [ ]:
df_true['Orbit Mean Density (kg/m^3)'].plot()

In [ ]:
df_pred

In [ ]:
df_pred_msis.set_index('Timestamp')['Density (kg/m3)'].plot()


###  Model by MSIS

In [ ]:
from atm_me import PersistenceMSIS, MSISPersistenceAtmosphere, PersistenceModel

In [ ]:
bad_id = []

In [ ]:
model = PersistenceModel(plot_trajectory=False)
for id in df_train.index:
    if os.path.exists('data_isaac/sat_density_pred_no_drag/density_pred'+ str(id) + '.csv'):
        continue
    else:
        bad_id.append(id)
        continue
    omni_data = omni[id].loc[:,['Timestamp', 'f10.7_index', 'ap_index_nT']]
    omni_data['Timestamp'] = pd.to_datetime(omni_data['Timestamp'])
    omni_data = omni_data.ffill()
    try:
        states, densities = model(omni_data,df_train.loc[id].to_dict())
        predictions = model._convert_to_df(states, densities)
        predictions.to_csv('data_isaac/sat_density_pred_no_drag/density_pred'+ str(id) + '.csv')
    except:
        print(id)

In [ ]:
from poliastro.core.elements import coe2rv

In [ ]:
from poliastro.constants.general import GM_earth

In [ ]:
coe2rv(GM_earth,6826.387247,0.003882,87.275306,144.135111,257.314389,102.383270)

In [ ]:
coe2rv(GM_earth,6824.556715,0.004397,87.269194,127.738000,86.817000,273.818611)

In [ ]:
df_train.loc[df_train['Latitude (deg)']<100]

In [ ]:
df_train.loc[bad_id]

In [ ]:
df_train.loc[df_train.index.isin(bad_id)==False]

In [ ]:
len(bad_id)


In [ ]:
from evaluation_me import DensityModelEvaluator

